In [52]:
import requests
import os
from dotenv import load_dotenv
from pathlib import Path
import json
import pandas as pd
import psycopg2

In [35]:
def load_secrets():
    load_dotenv()
    env_path = Path(".") / ".env"
    load_dotenv(dotenv_path=env_path)

    API_KEY = os.getenv("API_KEY")
    return API_KEY

In [36]:
API_KEY = load_secrets()

In [37]:
url = f"https://airquality.googleapis.com/v1/currentConditions:lookup?key={API_KEY}"

In [38]:
TIMEZONE_BASE_URL = "https://maps.googleapis.com/maps/api/timezone/json"

In [39]:
headers = {"Content-Type": "application/json"}

In [40]:
CAPITAIS = {
    "Rio Branco": (-9.97499, -67.8243),
    "Maceió": (-9.66599, -35.735),
    "Macapá": (0.0389, -51.0664),
    "Manaus": (-3.11866, -60.0212),
    "Salvador": (-12.9714, -38.5014),
    "Fortaleza": (-3.73186, -38.5267),
    "Brasília": (-15.7797, -47.9297),
    "Vitória": (-20.3155, -40.3128),
    "Goiânia": (-16.6786, -49.2539),
    "São Luís": (-2.52972, -44.3028),
    "Cuiabá": (-15.5989, -56.0949),
    "Campo Grande": (-20.4428, -54.6464),
    "Belo Horizonte": (-19.9167, -43.9345),
    "Belém": (-1.45502, -48.5024),
    "João Pessoa": (-7.115, -34.8631),
    "Curitiba": (-25.4278, -49.2731),
    "Recife": (-8.05, -34.9),
    "Teresina": (-5.08921, -42.8016),
    "Rio de Janeiro": (-22.9068, -43.1729),
    "Natal": (-5.795, -35.2094),
    "Porto Alegre": (-30.0331, -51.23),
    "Porto Velho": (-8.76194, -63.9039),
    "Boa Vista": (2.81972, -60.6733),
    "Florianópolis": (-27.5969, -48.5495),
    "São Paulo": (-23.5505, -46.6333),
    "Aracaju": (-10.9111, -37.0717),
    "Palmas": (-10.1675, -48.3277)
}

In [41]:
capitais_dict = {}
for capital, coords in CAPITAIS.items():
    response = requests.post(url=url,
                             json={"location": {"latitude": coords[0], "longitude": coords[1]}},
                             headers=headers)
    capitais_dict.update({capital : response.text})

In [44]:
capitais_dict


{'Rio Branco': '{\n  "dateTime": "2025-09-18T21:00:00Z",\n  "regionCode": "br",\n  "indexes": [\n    {\n      "code": "uaqi",\n      "displayName": "Universal AQI",\n      "aqi": 75,\n      "aqiDisplay": "75",\n      "color": {\n        "red": 0.43137255,\n        "green": 0.7764706,\n        "blue": 0.20392157\n      },\n      "category": "Good air quality",\n      "dominantPollutant": "o3"\n    }\n  ]\n}\n',
 'Maceió': '{\n  "dateTime": "2025-09-18T21:00:00Z",\n  "regionCode": "br",\n  "indexes": [\n    {\n      "code": "uaqi",\n      "displayName": "Universal AQI",\n      "aqi": 76,\n      "aqiDisplay": "76",\n      "color": {\n        "red": 0.4117647,\n        "green": 0.77254903,\n        "blue": 0.20392157\n      },\n      "category": "Good air quality",\n      "dominantPollutant": "o3"\n    }\n  ]\n}\n',
 'Macapá': '{\n  "dateTime": "2025-09-18T21:00:00Z",\n  "regionCode": "br",\n  "indexes": [\n    {\n      "code": "uaqi",\n      "displayName": "Universal AQI",\n      "aqi": 7

In [48]:
df

,0
Rio Branco,"{\n ""dateTime"": ""2025-09-18T21:00:00Z"",\n ""r..."
Maceió,"{\n ""dateTime"": ""2025-09-18T21:00:00Z"",\n ""r..."
Macapá,"{\n ""dateTime"": ""2025-09-18T21:00:00Z"",\n ""r..."
Manaus,"{\n ""dateTime"": ""2025-09-18T21:00:00Z"",\n ""r..."
Salvador,"{\n ""dateTime"": ""2025-09-18T21:00:00Z"",\n ""r..."
Fortaleza,"{\n ""dateTime"": ""2025-09-18T21:00:00Z"",\n ""r..."
Brasília,"{\n ""dateTime"": ""2025-09-18T21:00:00Z"",\n ""r..."
Vitória,"{\n ""dateTime"": ""2025-09-18T21:00:00Z"",\n ""r..."
Goiânia,"{\n ""dateTime"": ""2025-09-18T21:00:00Z"",\n ""r..."
São Luís,"{\n ""dateTime"": ""2025-09-18T21:00:00Z"",\n ""r..."


In [76]:
linhas = []
for cidade, data_json in capitais_dict.items():
    cidade_data = json.loads(data_json)
    idx = cidade_data["indexes"][0]
    linhas.append({
        "City": cidade,

            })
# print(linhas)
# cidade_data["indexes"][0].get('aqi')
# cidade_data["indexes"][0]["aqi"]
cidade_data["indexes"][0]["category"]

'Moderate air quality'

In [75]:
linhas = []
for cidade, data_json in capitais_dict.items():
    cidade_data = json.loads(data_json)
    idx = cidade_data["indexes"][0]
    linhas.append({
        "City": cidade,
        "Date/Time": cidade_data["dateTime"],
        "Local" : cidade_data["regionCode"],
        "AQI": cidade_data["indexes"][0]["aqi"],
        "Red": cidade_data["indexes"][0]["color"]["red"],
        "Green": cidade_data["indexes"][0]["color"]["green"],
        "Blue": cidade_data["indexes"][0]["color"]["blue"],
        "Label": cidade_data["indexes"][0]["category"],
        "Dominant Pollutant": cidade_data["dominantPollutant"]
    })

KeyError: 'dominantPollutant'

In [ ]:
for column, value in df:
    column = column
    

,0
Rio Branco,"{\n ""dateTime"": ""2025-09-18T15:00:00Z"",\n ""r..."
Maceió,"{\n ""dateTime"": ""2025-09-18T15:00:00Z"",\n ""r..."
Macapá,"{\n ""dateTime"": ""2025-09-18T15:00:00Z"",\n ""r..."
Manaus,"{\n ""dateTime"": ""2025-09-18T15:00:00Z"",\n ""r..."
Salvador,"{\n ""dateTime"": ""2025-09-18T15:00:00Z"",\n ""r..."
Fortaleza,"{\n ""dateTime"": ""2025-09-18T15:00:00Z"",\n ""r..."
Brasília,"{\n ""dateTime"": ""2025-09-18T15:00:00Z"",\n ""r..."
Vitória,"{\n ""dateTime"": ""2025-09-18T15:00:00Z"",\n ""r..."
Goiânia,"{\n ""dateTime"": ""2025-09-18T15:00:00Z"",\n ""r..."
São Luís,"{\n ""dateTime"": ""2025-09-18T15:00:00Z"",\n ""r..."


In [ ]:
# def preprocess_air_quality():
#     conn = psycopg2.connect(
#         host='Localhost',
#         dbname="meubanco",
#     )sdavadv